# LIBRERIAS

In [4]:
import requests
import pandas as pd
from io import StringIO
from string import ascii_uppercase as alfabeto
import pickle
from bs4 import BeautifulSoup
import re


# DATA EXTRACTION

In [ ]:
# Funciones 

def obtener_partidos(año,headers):
    url_all_worlds_cup= "https://es.wikipedia.org/wiki/Copa_Mundial_de_F%C3%BAtbol_de_"
    response=requests.get(f"{url_all_worlds_cup}{año}",headers=headers)
    response.raise_for_status()  # Si falla

    #Transformo en texto plano
    content=response.text
    soup=BeautifulSoup(content,'lxml')
    matches = soup.find_all("table", class_="vevent")
    local=[]
    resultado=[]
    visitante=[]
    dict_futbol={}
    for match in matches:
        equipos = match.select("td a[title^='Selección']")

        if len(equipos) < 2:
            continue

        equipo_local = equipos[0].get_text(strip=True)
        equipo_visitante = equipos[1].get_text(strip=True)
        marcador_tag = match.find("td", align="center")
        if not marcador_tag:
            continue

        marcador = marcador_tag.find("b")
        if not marcador:
            continue

        rdo = marcador.get_text(strip=True)
        rdo=re.search(r"\d+:\d+", rdo).group()
        resultado.append(rdo)

        local.append(equipo_local)
        visitante.append(equipo_visitante)

    dict_futbol={'local':local,'resultado':resultado,'visitante':visitante}
    df_futbol=pd.DataFrame(dict_futbol)
    df_futbol["año"]=año
    return df_futbol

def obtener_tablas_wiki(url,headers):

    response=requests.get(url,headers=headers)
    response.raise_for_status()  # Si falla
    #Transformo en texto plano
    html=response.text

    #Lo cargo en cache en string IO 
    df_list=pd.read_html(StringIO(html))
    print(f"Cantidad de Tablas {len(df_list)}")
    
    return df_list



def obtener_fixture(año, headers):
    import requests
    import pandas as pd
    from bs4 import BeautifulSoup

    url_all_worlds_cup = "https://es.wikipedia.org/wiki/Copa_Mundial_de_F%C3%BAtbol_de_"
    response = requests.get(f"{url_all_worlds_cup}{año}", headers=headers)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, 'lxml')
    matches = soup.find_all("table", class_="vevent")

    fechas = []
    locales = []
    partidos = []
    visitantes = []
    sedes = []

    for match in matches:
        # 👉 tomar SOLO la fila principal
        rows = match.find_all("tr")
        if not rows:
            continue

        cols = rows[0].find_all("td")
        if len(cols) < 5:
            continue

        fecha = cols[0].get_text(strip=True)
        local = cols[1].get_text(strip=True)
        partido = cols[2].get_text(strip=True)
        visitante = cols[3].get_text(strip=True)
        sede = cols[4].get_text(" ", strip=True)

        fechas.append(fecha)
        locales.append(local)
        partidos.append(partido)
        visitantes.append(visitante)
        sedes.append(sede)

    df_fixture = pd.DataFrame({
        "fecha": fechas,
        "local": locales,
        "partido": partidos,
        "visitante": visitantes,
        "sede": sedes
    })

    df_fixture["año"] = año
    return df_fixture


In [ ]:
headers = {
    "User-Agent": "PabloWikiBot/1.0 (contacto: pablolube@gmail.com)"
}

df_fixture=obtener_fixture(2026,headers)
df_fixture

## EXTRACCION DE MI DATAFRAME OBJETIVO

### Importo los Tabla de Grupos del mundial desde Wikipedia

In [ ]:
headers = {
    "User-Agent": "PabloWikiBot/1.0 (contacto: pablolube@gmail.com)"
}

url = "https://es.wikipedia.org/wiki/Copa_Mundial_de_F%C3%BAtbol_de_2026"

df_list=obtener_tablas_wiki(url,headers=headers)




Veo la información que me es relevante para el análisis

In [ ]:
j=0
for i in range(13,97,7):
    j=j+1
    print(f"GRUPO {j}")
    display(df_list[i])

Cambio el nombre, por el nombre real de cada grupo y lo guardo en un diccionario

In [ ]:
alfabeto

In [ ]:
dict_tablas={}
for letra,i in zip(alfabeto,range(13,97,7)):
    df=df_list[i]
    dict_tablas[f"Grupo {letra}"]=df

print(dict_tablas)

In [ ]:
with open("Data\grupos_mundial.pkl", "wb") as f:
    pickle.dump(dict_tablas, f)

## Extraccion de los partidos historicos 

In [ ]:
# Listado de años de mundiales pasados
year_world_cup_mundial = [

 1930, 1934, 1938,
    1950, 1954, 1958, 1962, 1966,
    1970, 1974, 1978, 1982,
    1986, 1990, 1994, 1998,
    2002, 2006, 2010, 2014,
    2018, 2022]

# Rest
headers = {
    "User-Agent": "PabloWikiBot/1.0 (contacto: pablolube@gmail.com)"
}

dfs=[]
# Recorro años
for año in year_world_cup_mundial:
    #Obtengo el df
    df=obtener_partidos(año,headers)
    #Lo agrego al df final en forma de lista
    dfs.append(df)

# Concateno las listas en un solo df
df_world_cup_historic = pd.concat(dfs, ignore_index=True)

#Guardo en CSV
df_world_cup_historic.to_csv("World_Cup_Results.csv",index=False)

In [ ]:

pd.set_option("display.max_rows", None)
df_world_cup_historic

In [ ]:
from selenium import webdriver

driver = webdriver.Chrome()
driver.get("https://www.google.com")

input("Enter para cerrar")
driver.quit()


## Importo Fixture 2026

In [ ]:
headers = {
    "User-Agent": "PabloWikiBot/1.0 (contacto: pablolube@gmail.com)"
}

df_fixture=obtener_fixture(2026,headers)
df_fixture

In [ ]:
df_fixture.to_csv("fifa_worldcup_fixture.csv",index=False)

# LIMPIEZA DE DATOS

Cargo Datasets

In [ ]:
# Cargo dataset
df_data_historical=pd.read_csv('Data\RAW_FIFA_World_Cup_historic.csv')

Limpio Data Historica

In [27]:
df_data_historical

,local,resultado,visitante,año
0,Francia,4:1,México,1930
1,Argentina,1:0,Francia,1930
2,Chile,3:0,México,1930
3,Chile,1:0,Francia,1930
4,Argentina,6:3,México,1930
...,...,...,...,...
958,Inglaterra,1:2,Francia,2022
959,Argentina,3:0,Croacia,2022
960,Francia,2:0,Marruecos,2022
961,Croacia,2:1,Marruecos,2022


In [28]:
# Separar columna resultado en dos columnas numéricas
df_data_historical[['goles_local', 'goles_visitante']] = (
    df_data_historical['resultado']
        .str.strip()                    # elimina espacios
        .str.split(':', expand=True)
        .astype(int)
)

# Ordenar por año
df_data_historical.sort_values('año', inplace=True)

# Eliminar columna original
df_data_historical.drop(columns='resultado', inplace=True)

# Renombrar columnas
df_data_historical.rename(columns={
    "local": "HomeTeam",
    "visitante": "AwayTeam",
    "año": "Year",
    "goles_local": "HomeGoals",
    "goles_visitante": "AwayGoals"
}, inplace=True)


In [30]:
df_data_historical.info()

<class 'pandas.DataFrame'>
RangeIndex: 963 entries, 0 to 962
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   HomeTeam   963 non-null    str  
 1   AwayTeam   963 non-null    str  
 2   Year       963 non-null    int64
 3   HomeGoals  963 non-null    int64
 4   AwayGoals  963 non-null    int64
dtypes: int64(3), str(2)
memory usage: 37.7 KB


In [31]:
df_data_historical.to_csv("clean_fifa_worldcup_matches.csv",index=False)

## Limpieza Fixture

In [ ]:
df_fixture=pd.read_csv("Data\RAW_FIFA_World_CUP_Fixture_2026.csv")
df_fixture

,fecha,local,partido,visitante,sede,año
0,11 de junio de 2026,México,Partido 1,Sudáfrica,"Estadio Azteca , Ciudad de México",2026
1,11 de junio de 2026,Corea del Sur,Partido 2,A4,"Estadio Chivas , Guadalajara",2026
2,18 de junio de 2026,A4,Partido 25,Sudáfrica,"Mercedes-Benz Stadium , Atlanta",2026
3,18 de junio de 2026,México,Partido 28,Corea del Sur,"Estadio Chivas , Guadalajara",2026
4,24 de junio de 2026,A4,Partido 53,México,"Estadio Azteca , Ciudad de México",2026
...,...,...,...,...,...,...
99,11 de julio de 2026,Ganador Partido 95,Partido 100,Ganador Partido 96,"Arrowhead Stadium , Kansas City",2026
100,14 de julio de 2026,Ganador Partido 97,Partido 101,Ganador Partido 98,"AT&T Stadium , Dallas",2026
101,15 de julio de 2026,Ganador Partido 99,Partido 102,Ganador Partido 100,"Mercedes-Benz Stadium , Atlanta",2026
102,18 de julio de 2026,Perdedor Partido 101,Partido 103,Perdedor Partido 102,"Hard Rock Stadium , Miami",2026


In [39]:
df_fixture.drop(columns=["fecha","sede"],inplace=True)
df_fixture.rename(columns={"local":"home","partido":"score","visitante":"away","año":"year"},inplace=True)
df_fixture
df_fixture.to_csv("Clean_fifa_worldcup_fixture.csv", index=False)


In [34]:
df_fixture.info()

<class 'pandas.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   home    104 non-null    str  
 1   score   104 non-null    str  
 2   away    104 non-null    str  
 3   year    104 non-null    int64
dtypes: int64(1), str(3)
memory usage: 3.4 KB
